In [1]:
#!pip install sentence-transformers transformers torch scikit-learn pandas

In [2]:
# Sentence-BERT (SBERT)
from sentence_transformers import SentenceTransformer
import numpy as np

# Initialize SBERT model
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')  # A lightweight and efficient model
# Your industry description
industry_text = "industry data warehouse and data for businesses"

# Texts from different websites
website_texts = [
    "We are the best business data seller in the country",
    "We can give you the best warehouse for you in the country",
    "Look no further for your warehouse needs",
    # Add more website texts
]

# Generate embeddings
industry_embedding_sbert = sbert_model.encode(industry_text)
website_embeddings_sbert = sbert_model.encode(website_texts)
from sklearn.metrics.pairwise import cosine_similarity

# Compute similarities
similarities_sbert = cosine_similarity([industry_embedding_sbert], website_embeddings_sbert)[0]


ModuleNotFoundError: No module named 'sentence_transformers'

In [3]:
# LaBSE (Language-agnostic BERT Sentence Embedding)
from sentence_transformers import SentenceTransformer

# Initialize LaBSE model
labse_model = SentenceTransformer('sentence-transformers/LaBSE')
# Generate embeddings
industry_embedding_labse = labse_model.encode(industry_text)
website_embeddings_labse = labse_model.encode(website_texts)
# Compute similarities
similarities_labse = cosine_similarity([industry_embedding_labse], website_embeddings_labse)[0]


C:\Users\miken\anaconda3\envs\mlpy\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# TF-IDF with Cosine Similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine all texts for fitting the vectorizer
all_texts = [industry_text] + website_texts

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(all_texts)
# Industry vector is the first vector
industry_vector_tfidf = tfidf_matrix[0]

# Website vectors
website_vectors_tfidf = tfidf_matrix[1:]
# Compute cosine similarities
similarities_tfidf = cosine_similarity(industry_vector_tfidf, website_vectors_tfidf)[0]


In [5]:
# Comparing the Methods
import pandas as pd

# Create a DataFrame to store similarity scores
df_results = pd.DataFrame({
    'Website': [f"Website {i+1}" for i in range(len(website_texts))],
    'SBERT Similarity': similarities_sbert,
    'LaBSE Similarity': similarities_labse,
    'TF-IDF Similarity': similarities_tfidf
})

# Display the DataFrame
print(df_results)

     Website  SBERT Similarity  LaBSE Similarity  TF-IDF Similarity
0  Website 1          0.555404          0.442938           0.285859
1  Website 2          0.418224          0.291702           0.143347
2  Website 3          0.518358          0.312297           0.118696


In [7]:
# Find the best match for each method
best_sbert = df_results.loc[df_results['SBERT Similarity'].idxmax()]
best_labse = df_results.loc[df_results['LaBSE Similarity'].idxmax()]
best_tfidf = df_results.loc[df_results['TF-IDF Similarity'].idxmax()]

print("\nBest Matches per Method:")
print(f"SBERT: {best_sbert['Website']} with similarity {best_sbert['SBERT Similarity']:.4f}")
print(f"LaBSE: {best_labse['Website']} with similarity {best_labse['LaBSE Similarity']:.4f}")
print(f"TF-IDF: {best_tfidf['Website']} with similarity {best_tfidf['TF-IDF Similarity']:.4f}")
# Count the number of methods that selected each website as the best match
best_matches = df_results[['SBERT Similarity', 'LaBSE Similarity', 'TF-IDF Similarity']].idxmax()

# Map index to website names
best_matches = best_matches.map(lambda x: df_results.loc[x, 'Website'])

# Count occurrences
consensus = best_matches.value_counts()

print("\nConsensus on Best Match:")
print(consensus)


Best Matches per Method:
SBERT: Website 1 with similarity 0.5554
LaBSE: Website 1 with similarity 0.4429
TF-IDF: Website 1 with similarity 0.2859

Consensus on Best Match:
Website 1    3
Name: count, dtype: int64
